In [1]:
# TODO:
# * how to deal with new ...
# * adding time
# * adding popularity

# Recommender system demo

In [35]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

In [3]:
POPULARITY_COEFF = 1
TIME_COEFF = 1

In [4]:
tracks_db = pd.read_csv('../data/features.csv', index_col='track_id')
tracks_db.head()

,title,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artists_ids,genres
track_id,,,,,,,,,,,,,,,,
0aQA9DP54h37OevE7hRc2a,NaN,0.653,0.796,7,-4.050,0,0.2450,0.2400,0.00000,0.1080,0.405,160.026,252712,4,"[""6fOMl44jA4Sp5b9PpYCkzz""]","[[""hip hop"", ""pop rap"", ""rap""]]"
4ixBAI0E1M6df9mP27etZK,NaN,0.731,0.494,4,-9.567,1,0.0467,0.2010,0.00000,0.1600,0.477,120.045,228056,4,"[""46VwdSiSccsgzNhJP9sGhQ"", ""2272CVpxVafY3L0UHu...","[[""icelandic hip hop"", ""icelandic pop""]]"
3jO5IAvslWxaTxswokg66i,NaN,0.859,0.464,3,-7.328,1,0.2890,0.2080,0.00000,0.1140,0.370,134.966,195453,4,"[""2LRoIwlKmHjgvigdNGBHNo"", ""5J7rXWjtn5HzUkJ4Je...","[[""colombian pop"", ""latin"", ""pop reggaeton"", ""..."
5iS3TEmmRcDJU2pTJQ98zB,NaN,0.482,0.617,10,-10.288,0,0.0869,0.7750,0.58300,0.1070,0.350,174.040,300861,4,"[""3MaQwUQBigrjcMYHk8bDKm"", ""5Y24oYaTX3VOvtAcG2...","[[""finnish dance pop"", ""finnish hip hop"", ""fin..."
29xPIFa0othUcFrEGyJCZu,NaN,0.824,0.684,1,-7.185,1,0.4070,0.0236,0.00853,0.0762,0.509,141.143,186973,4,"[""2s7fZ34RK8RUJQ2ewsh6I5"", ""3tTvSeZiFDP3CY5EdP...","[[""francoton"", ""french hip hop"", ""pop urbaine""..."


### Version 1: using pure audio features only

In [5]:
feature_names = [
    'danceability', 'energy', 'key', 'loudness',	
    'mode',	'speechiness', 'acousticness', 'instrumentalness',
    'liveness',	'valence', 'tempo', 'duration_ms', 'time_signature']
tracks_db = tracks_db[feature_names]
tracks_db.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
track_id,,,,,,,,,,,,,
0aQA9DP54h37OevE7hRc2a,0.653,0.796,7,-4.050,0,0.2450,0.2400,0.00000,0.1080,0.405,160.026,252712,4
4ixBAI0E1M6df9mP27etZK,0.731,0.494,4,-9.567,1,0.0467,0.2010,0.00000,0.1600,0.477,120.045,228056,4
3jO5IAvslWxaTxswokg66i,0.859,0.464,3,-7.328,1,0.2890,0.2080,0.00000,0.1140,0.370,134.966,195453,4
5iS3TEmmRcDJU2pTJQ98zB,0.482,0.617,10,-10.288,0,0.0869,0.7750,0.58300,0.1070,0.350,174.040,300861,4
29xPIFa0othUcFrEGyJCZu,0.824,0.684,1,-7.185,1,0.4070,0.0236,0.00853,0.0762,0.509,141.143,186973,4


In [6]:
cat_features = ['key', 'mode', 'time_signature']
for cat_feature in cat_features:
    dummies = pd.get_dummies(tracks_db[cat_feature], prefix=f'{cat_feature}_')
    tracks_db = pd.concat([tracks_db, dummies], axis=1)

tracks_db.drop(columns=cat_features, inplace=True)
tracks_db.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,key__9,key__10,key__11,mode__0,mode__1,time_signature__0,time_signature__1,time_signature__3,time_signature__4,time_signature__5
track_id,,,,,,,,,,,,,,,,,,,,,
0aQA9DP54h37OevE7hRc2a,0.653,0.796,-4.050,0.2450,0.2400,0.00000,0.1080,0.405,160.026,252712,...,0,0,0,1,0,0,0,0,1,0
4ixBAI0E1M6df9mP27etZK,0.731,0.494,-9.567,0.0467,0.2010,0.00000,0.1600,0.477,120.045,228056,...,0,0,0,0,1,0,0,0,1,0
3jO5IAvslWxaTxswokg66i,0.859,0.464,-7.328,0.2890,0.2080,0.00000,0.1140,0.370,134.966,195453,...,0,0,0,0,1,0,0,0,1,0
5iS3TEmmRcDJU2pTJQ98zB,0.482,0.617,-10.288,0.0869,0.7750,0.58300,0.1070,0.350,174.040,300861,...,0,1,0,1,0,0,0,0,1,0
29xPIFa0othUcFrEGyJCZu,0.824,0.684,-7.185,0.4070,0.0236,0.00853,0.0762,0.509,141.143,186973,...,0,0,0,0,1,0,0,0,1,0


In [38]:
scaler = MinMaxScaler()
tracks_db_scaled = pd.DataFrame(
    scaler.fit_transform(tracks_db.to_numpy()), columns=tracks_db.columns)
tracks_db_scaled.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,key__9,key__10,key__11,mode__0,mode__1,time_signature__0,time_signature__1,time_signature__3,time_signature__4,time_signature__5
0,0.662944,0.795214,0.836288,0.253623,0.241449,0.000000,0.096867,0.409505,0.695481,0.175277,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.742132,0.492049,0.684643,0.048344,0.202213,0.000000,0.150113,0.482305,0.521722,0.155861,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.872081,0.461934,0.746186,0.299172,0.209256,0.000000,0.103010,0.374115,0.586569,0.130187,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.489340,0.615524,0.664825,0.089959,0.779678,0.588889,0.095843,0.353893,0.756387,0.213194,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.836548,0.682782,0.750117,0.421325,0.023742,0.008616,0.064305,0.514661,0.613415,0.123509,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [39]:
tracks_user = tracks_db.sample(5)
tracks_user.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,key__9,key__10,key__11,mode__0,mode__1,time_signature__0,time_signature__1,time_signature__3,time_signature__4,time_signature__5
track_id,,,,,,,,,,,,,,,,,,,,,
2L0hkkXKyXOlxQNJkn8k4Y,0.662,0.637,-6.689,0.1150,0.397000,0.004940,0.0924,0.727,132.073,156533,...,0,0,0,1,0,0,0,0,1,0
7HGTkn2aX7MNdKs7nV2xBt,0.509,0.987,-3.198,0.0886,0.000081,0.000928,0.1190,0.647,138.039,197160,...,0,1,0,1,0,0,0,0,1,0
1K5KBOgreBi5fkEHvg5ap3,0.795,0.574,-6.903,0.4870,0.067000,0.000000,0.1500,0.537,142.053,237918,...,0,0,0,1,0,0,0,0,1,0
09ts3GnICqYEU5PkQCpJK3,0.707,0.632,-7.031,0.0789,0.103000,0.000011,0.0841,0.372,97.040,278893,...,0,1,0,0,1,0,0,0,1,0
2xI4CmkT6Bl4s5b4qggcgU,0.328,0.631,-4.346,0.0476,0.260000,0.000000,0.2690,0.546,156.559,185237,...,0,0,1,1,0,0,0,0,1,0


In [40]:
def get_similarites(tracks_db_scaled, tracks_user, scaler):
    user_vector = tracks_user.to_numpy()
    user_vector_scaled = scaler.transform(user_vector)
    user_vector_scaled = np.sum(user_vector_scaled, axis=0, keepdims=True)

    return cosine_similarity(
        tracks_db_scaled, user_vector_scaled)

In [41]:
similarities = get_similarites(tracks_db_scaled, tracks_user, scaler)
track_db = tracks_db.assign(similarity=similarities)
track_db.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,key__10,key__11,mode__0,mode__1,time_signature__0,time_signature__1,time_signature__3,time_signature__4,time_signature__5,similarity
track_id,,,,,,,,,,,,,,,,,,,,,
0aQA9DP54h37OevE7hRc2a,0.653,0.796,-4.050,0.2450,0.2400,0.00000,0.1080,0.405,160.026,252712,...,0,0,1,0,0,0,0,1,0,0.862464
4ixBAI0E1M6df9mP27etZK,0.731,0.494,-9.567,0.0467,0.2010,0.00000,0.1600,0.477,120.045,228056,...,0,0,0,1,0,0,0,1,0,0.708361
3jO5IAvslWxaTxswokg66i,0.859,0.464,-7.328,0.2890,0.2080,0.00000,0.1140,0.370,134.966,195453,...,0,0,0,1,0,0,0,1,0,0.711182
5iS3TEmmRcDJU2pTJQ98zB,0.482,0.617,-10.288,0.0869,0.7750,0.58300,0.1070,0.350,174.040,300861,...,1,0,1,0,0,0,0,1,0,0.869594
29xPIFa0othUcFrEGyJCZu,0.824,0.684,-7.185,0.4070,0.0236,0.00853,0.0762,0.509,141.143,186973,...,0,0,0,1,0,0,0,1,0,0.729934


In [42]:
track_db.sort_values('similarity', ascending=False).iloc[:10]

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,...,key__10,key__11,mode__0,mode__1,time_signature__0,time_signature__1,time_signature__3,time_signature__4,time_signature__5,similarity
track_id,,,,,,,,,,,,,,,,,,,,,
30F6ieEtboBJPoE92epXMO,0.783,0.822,-3.778,0.2370,0.1910,0.000009,0.1950,0.631,139.990,210232,...,1,0,1,0,0,0,0,1,0,0.952368
6ZR7rT7vfwtY0EqyLeyAaO,0.687,0.872,-3.013,0.0983,0.1260,0.008390,0.0906,0.706,179.991,190354,...,1,0,1,0,0,0,0,1,0,0.952084
4iYxrnTvYsTtyphOTqTvkh,0.787,0.842,-4.026,0.2500,0.1170,0.000000,0.1790,0.657,138.159,231428,...,1,0,1,0,0,0,0,1,0,0.951795
2FSVMyqLldC36p0deIYvI9,0.746,0.884,-3.348,0.3300,0.1380,0.000000,0.1460,0.664,140.100,264416,...,1,0,1,0,0,0,0,1,0,0.951689
5mhnyYQ0h9PcICiVj3lhYI,0.703,0.844,-4.648,0.1400,0.1580,0.000000,0.1680,0.627,139.996,271019,...,1,0,1,0,0,0,0,1,0,0.951498
0oY8xK12tJkYAskQXXQsgj,0.817,0.874,-0.091,0.2150,0.0353,0.000136,0.1570,0.908,160.009,403687,...,1,0,1,0,0,0,0,1,0,0.951262
4b0uPErJSib9k0NwJ8lkRc,0.828,0.734,-3.928,0.2010,0.2830,0.000000,0.0965,0.681,168.033,175303,...,1,0,1,0,0,0,0,1,0,0.951136
3m9zvjbilaBUMKNFcROBfF,0.815,0.828,-3.278,0.1630,0.0827,0.000002,0.1340,0.882,176.086,192324,...,1,0,1,0,0,0,0,1,0,0.951112
07KbI05QnjX8IbDBDRySxR,0.769,0.874,-2.951,0.1420,0.0403,0.000000,0.1000,0.670,145.000,222040,...,1,0,1,0,0,0,0,1,0,0.950958


### Version 2: adding time

In [ ]:
# jest release date, dodać losowe do bazy
# sim = audio_sim + time_sim